In [170]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

URL = 'https://twilight-imperium.fandom.com/wiki/Flagship'

# Fetch the webpage
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Locate Each Faction Heading
headings = soup.find_all('h3')
clean_headings = []
for heading in headings:
    if heading.find('span', class_='mw-headline'):
        clean_headings.append(heading)
headings = clean_headings

# Locate Each Heading Span
spans = []
for heading in headings:
    span = heading.find('span', class_='mw-headline')
    if span:
        spans.append(span)

# Locate Faction Name from each Span
factions = []
for span in spans:
    name = span.get_text(strip=True).strip('[]')
    factions.append(name)

# Locate Each Faction Table from each Heading
tables = []
for heading in headings:
    table = heading.find_next_sibling()
    tables.append(table)
# Currently missing The Nomad's flagship v2

# Scrape Data for Each Flagship
all_flagships = []
for table in tables:
    rows = table.find_all('tr')
    name = rows[0].get_text(strip=True)
    abilities = rows[1].get_text().lstrip('\n').rstrip('\n').split('\n')
    ability = abilities[0]

    bonus_abilities = abilities[1:]
    bonus_abilities = ' & '.join(bonus_abilities)

    cells = rows[2].find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    cost = row_data[0]
    combat = row_data[1]
    move = row_data[2]
    capacity = row_data[3]

    flagship_stats = {'Name': name, 'Ability': ability, 'Bonus_Abilities': bonus_abilities, 'Cost': cost, 'Combat': combat, 'Move': move, 'Capacity': capacity}
    all_flagships.append(flagship_stats)

flagships_df = pd.DataFrame(all_flagships, index=factions)
flagships_df

,Name,Ability,Bonus_Abilities,Cost,Combat,Move,Capacity
The Arborec,Duha Menaimon,"After you activate this system, you may produc...",Sustain Damage,8,7 (x2),1,5
The Argent Flight,Quetzecoatl,Other players cannot use SPACE CANNON against ...,Sustain Damage,8,7 (x2),1,3
The Barony of Letnev,Arc Secundus,Other players' units in this system lose PLANE...,"At the start of each space combat round, repai...",8,5 (x2),1,3
The Clan of Saar,Son of Ragh,Anti-Fighter Barrage 6 (x4),Sustain Damage,8,5 (x2),1,3
The Council Keleres,Artemiris,Other players must spend 2 influence to activa...,Sustain Damage,8,7 (x2),1,6
The Embers of Muaat,The Inferno,ACTION: Spend 1 token from your strategy pool ...,Sustain Damage,8,5 (x2),1,3
The Emirates of Hacan,Wrath of Kenara,After you roll a die during space combat in th...,Sustain Damage,8,7 (x2),1,3
The Empyrean,Dynamo,After any player's unit in this system or an a...,Sustain Damage,8,5(x2),1,3
The Federation of Sol,Genesis,"At the end of the status phase, place 1 infant...",Sustain Damage,8,5 (x2),1,12
The Ghosts of Creuss,Hil Colish,This ship's system contains a delta wormhole,"During movement, this ship may move before or ...",8,5,1,3


In [173]:
ships_list = ['Carrier', 'Cruiser', 'Destroyer', 'Dreadnought', 'Fighter', 'War_Sun']
ships = []

for ship in ships_list:

    URL = f'https://twilight-imperium.fandom.com/wiki/{ship}'

    # Fetch the webpage
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    tables = soup.find_all('table', class_="article-table")


    for table in tables:
        name = table.find('th').get_text(strip=True)
        table = pd.read_html(StringIO(str(table)))
        table = table[0]
        if table.iloc[0,4] == 'Upg.':
            ability = table.iloc[0,3]
            cost = table.iloc[3,0]
            combat = table.iloc[3,1]
            move = table.iloc[3,2]
            capacity = table.iloc[3,3]
        elif table.iloc[0,0] == ('Req.' or 'Cost'):
            ability = table.iloc[0,1]
            cost = table.iloc[3,1]
            combat = table.iloc[3,2]
            move = table.iloc[3,3]
            capacity = table.iloc[3,4]

        ship = {'Name': name, 'Cost': cost, 'Combat': combat, 'Move': move, 'Capacity': capacity, 'Ability': ability}
        ships.append(ship)

df = pd.DataFrame(ships)
df

,Name,Cost,Combat,Move,Capacity,Ability
0,Carrier,3,9,1^,4^,NaN
1,Carrier II,3,9,2,6,NaN
2,Advanced Carrier,3,9,1^,6^,NaN
3,Advanced Carrier II,3,9,2,8,Sustain Damage
4,Cruiser,2,7^,2^,-^,NaN
5,Cruiser II,2,6,3,1,NaN
6,Saturn Engine I,2,7^,2^,1^,NaN
7,Saturn Engine II,2,6,3,2,Sustain Damage
8,Destroyer,1,9^,2,NaN,Anti-Fighter Barrage 9 (x2)
9,Destroyer II,1,8,2,NaN,Anti-Fighter Barrage 6 (x3)


In [174]:
ships_list = ['Infantry', 'Mechs']
ships = []

for ship in ships_list:

    URL = f'https://twilight-imperium.fandom.com/wiki/{ship}'

    # Fetch the webpage
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    tables = soup.find_all('table', class_="article-table")


    for table in tables:
        name = table.find('th').get_text(strip=True)
        table = pd.read_html(StringIO(str(table)))
        table = table[0]
        if table.iloc[0,4] == 'Upg.':
            ability = table.iloc[0,3]
            cost = table.iloc[3,0]
            combat = table.iloc[3,1]
            move = table.iloc[3,2]
            capacity = table.iloc[3,3]
        elif table.iloc[0,0] == ('Req.' or 'Cost'):
            ability = table.iloc[0,1]
            cost = table.iloc[3,1]
            combat = table.iloc[3,2]
            move = table.iloc[3,3]
            capacity = table.iloc[3,4]

        ship = {'Name': name, 'Cost': cost, 'Combat': combat, 'Move': move, 'Capacity': capacity, 'Ability': ability}
        ships.append(ship)

df = pd.DataFrame(ships)
df

IndexError: index 4 is out of bounds for axis 0 with size 2

In [172]:
ships = {
    "Carrier": {"Cost": 3, "Combat": 9, "Move": 1, "Capacity": 4},
    "Carrier II": {"Cost": 3, "Combat": 9, "Move": 2, "Capacity": 6},
    "Advanced Carrier"
    "Cruiser": {"Cost": 2, "Combat": 7, "Move": 2, "Capacity": 1},
    "Destroyer": {"Cost": 1, "Combat": 9, "Move": 2, "Capacity": 0},
    "Dreadnought": {"Cost": 4, "Combat": 5, "Move": 1, "Capacity": 1},
}
